# All Call Graph

In [ ]:
import networkx as nx

from qualtran_dev_tools.bloq_finder import get_bloq_examples, get_bloq_classes
from qualtran_dev_tools.all_call_graph import get_all_call_graph, call_graph_to_class_graph, ClassGraphGraphviz

In [ ]:
bes = get_bloq_examples()

In [ ]:
g = get_all_call_graph(bes)

In [ ]:
from qualtran.drawing import show_call_graph
show_call_graph(g)

In [ ]:
for x in nx.simple_cycles(g):
    print(x)

In [ ]:
# with open('all-call.svg', 'wb') as f:
#     f.write(GraphvizCounts(g).get_svg_bytes())

In [ ]:
# import pydot
# def get_graph(self):
#     """Get the pydot graph."""
#     graph = pydot.Dot('classes', graph_type='digraph', rankdir='TB', ranksep=0.8)
#     self.add_nodes(graph)
#     self.add_edges(graph)
#     return graph
# ClassGraphGraphviz.get_graph = get_graph

In [ ]:
g2 = call_graph_to_class_graph(g)
ClassGraphGraphviz(g2).get_svg()

In [ ]:
for x in nx.simple_cycles(g2):
    print(x)

In [ ]:
with open('class-graph.svg', 'wb') as f:
    f.write(ClassGraphGraphviz(g2).get_svg_bytes())

In [ ]:
g3 = nx.subgraph(g2, max(nx.weakly_connected_components(g2), key=len))
ClassGraphGraphviz(g3).get_svg()

In [ ]:
bcls = get_bloq_classes()

from qualtran.bloqs.util_bloqs import Split, Join, Partition, Cast, Allocate, Free, ArbitraryClifford

for k in [Split, Join, Partition, Cast, Allocate, Free, ArbitraryClifford]:
    bcls.remove(k)

bcls = [k for k in bcls if not k.__module__.startswith('qualtran.bloqs.for_testing')]

In [ ]:
import pandas as pd

records = []
for bcl in bcls:
    records += [{
        'bloq_cls': bcl.__name__,
        'exists': bcl in g2.nodes,
        'connected': bcl in g3.nodes,
    }]

df = pd.DataFrame(records)

def colors(x):
    if x:
        return 'background-color: lightgreen'
    return 'background-color:pink'

df.style.map(colors, ['exists', 'connected'])

In [ ]:
df2 = df[(~df['exists']|~df['connected'])]
df2.style.map(colors, ['exists', 'connected'])